Para comenzar, es necesario instalar el paquete `sympy` antes de poder utilizarlo en tu programa de Python. A continuación, se presentan varias formas de instalar el paquete `sympy`:

1. **Instalación utilizando pip:**

   ```bash
   pip install sympy
   ```

   Este comando instalará el paquete `sympy` a través del administrador de paquetes de Python, `pip`. Asegúrate de tener `pip` instalado y configurado correctamente en tu entorno.

2. **Instalación utilizando conda:**

   ```bash
   conda install sympy
   ```

   Si utilizas el administrador de paquetes `conda`, puedes instalar `sympy` ejecutando este comando. Asegúrate de tener `conda` instalado y configurado en tu entorno.

3. **Instalación mediante entornos de desarrollo integrados (IDEs):**

   Algunos entornos de desarrollo integrados (IDEs), como PyCharm o Anaconda Navigator, ofrecen interfaces gráficas que permiten instalar paquetes de Python de forma sencilla. Puedes buscar y seleccionar `sympy` desde la interfaz gráfica y seguir los pasos proporcionados para completar la instalación.

Es recomendable utilizar la opción que mejor se adapte a tu entorno y flujo de trabajo. Una vez instalado el paquete `sympy`, podrás importarlo en tu programa de Python y utilizarlo para realizar cálculos simbólicos y algebraicos.

In [113]:
from Py_Matrix import  Matrix
from functools import reduce
from sympy import  expand,Poly

In [114]:
class OpenMethods:
    def __init__(self,error) -> None:
        self.error = error


In [115]:
def parseRow(line):
    """
    Convierte una línea de texto en una lista de números de punto flotante.

    Args:
        line (str): Línea de texto que contiene los números separados por comas.

    Returns:
        list: Lista de números de punto flotante obtenidos al convertir la línea.

    Raises:
        ValueError: Si ocurre un error al convertir los caracteres en números de punto flotante.
    """
    try:
        numbers = line.replace("\n", "").split(',')
        return list(map(lambda char: float(char), numbers))
    except ValueError as e:
        raise ValueError(
            "Error al convertir los caracteres en números de punto flotante: " + str(e))


def getMatrixFromCsv(pathFile):
    """
    Carga una matriz desde un archivo CSV y la devuelve como un objeto Matrix de Py_Matrix.

    Args:
        pathFile (str): Ruta del archivo CSV.

    Returns:
        Matrix: Objeto Matrix de Py_Matrix que representa la matriz cargada desde el archivo CSV.

    Raises:
        IOError: Si ocurre un error al abrir o leer el archivo CSV.
    """
    try:
        with open(pathFile, "r") as f:
            matrixOfNumbers = [parseRow(row) for row in f.readlines()]
            vectorRow = [number for row in matrixOfNumbers for number in row]
            return Matrix(vectorRow)
    except IOError as e:
        raise IOError("Error al abrir o leer el archivo CSV: " + str(e))


def getPolynomial(matrix: Matrix):
    """
    Obtiene el polinomio característico de una matriz y lo devuelve como un objeto polinomial.

    Args:
        matrix (Matrix): Matriz de la cual se obtendrá el polinomio característico.

    Returns:
        Poly: Objeto polinomial que representa el polinomio característico de la matriz.
    """
    return expand(matrix.characteristic_polynomial()).as_poly()


def isVariation(a, b):
    """
    Determina si hay un cambio de signo entre los números a y b.

    Args:
        a (float): Primer número.
        b (float): Segundo número.

    Returns:
        int: 1 si hay un cambio de signo entre a y b, 0 en caso contrario.

    Examples:
        >>> isVariation(2, -3)
        1
        >>> isVariation(4, 4)
        0
    """
    if max(abs(a), abs(b)) < abs(a - b):
        return 1
    return 0



def reverse(array):
    """
    Invierte el orden de los elementos en un arreglo y lo devuelve.

    Args:
        array (list): Arreglo que se desea invertir.

    Returns:
        list: Arreglo con los elementos invertidos.

    Examples:
        >>> reverse([1, 2, 3, 4])
        [4, 3, 2, 1]
        >>> reverse(['a', 'b', 'c'])
        ['c', 'b', 'a']
    """
    return array[::-1]

def adaptGrades(monoms):
    """
    Adapta los monomios de un polinomio para obtener una lista de los grados de cada monomio.

    Args:
        monoms (list): Lista de monomios obtenidos a partir del método `monoms` de la clase `Poly` de Sympy.

    Returns:
        list: Lista de los grados de cada monomio en el mismo orden que los monomios.

    Examples:
        >>> p = Poly('x**2 + 2*x + 1', domain='QQ')
        >>> monoms = p.monoms()
        >>> adaptGrades(monoms)
        [2, 1, 0]
    """
    return [x[0] for x in monoms]



def adaptCoefficients(coefficients):
    """
    Adapta los coeficientes de un polinomio para obtener una lista de coeficientes no nulos.

    Args:
        coefficients (list): Lista de coeficientes obtenidos a partir del método `all_coeffs` de un objeto polinomial.

    Returns:
        list: Lista de coeficientes no nulos en el mismo orden que los coeficientes originales.

    Examples:
        >>> p = Poly('3*x**2 + 0*x + 4', domain='QQ')
        >>> coefficients = p.all_coeffs()
        >>> adaptCoefficients(coefficients)
        [3, 4]
    """
    return list(filter(lambda a_i: a_i != 0.0, coefficients))


$$\text{Proposición 4.1 } \textit{Sea } p(z):=a_0+...+a_nz^n \textit{ con } a_n \neq 0 \newline
\textit{Si } z_0 \textit{ es una raiz de p entonces: } \newline
\lvert z_0 \rvert \leq \max{1,\frac{\lvert a_0 \rvert + \lvert a_1 \rvert+...+\lvert a_{n-1} \rvert}{\lvert a_n \rvert}}$$

In [116]:
def upperBoundForComplexRoots(poly: Poly):
    """
    Calcula el límite superior de las raíces complejas de un polinomio. Es un caso mas general.
    Si hablamos solo de raices reales solo sera el valor absoluto. Con esto sabemos que todas
    las raices reales del polinomio, si existen, estan entre (-x0,x0) siendo x0 el caso real de
    z0

    Args:
        poly (Poly): Objeto polinomial.

    Returns:
        float: Límite superior de las raíces complejas.
    Examples:
        >>> p = Poly('x**4-10*x**3+35*x**2-50*x+24')
        >>> 119 -> Todas las raices reales estan en (-119,119)
    """
    coefficients = reverse(poly.all_coeffs())
    lenCoeff = len(coefficients)
    an = coefficients[lenCoeff - 1]
    return max(1, reduce(lambda acc, new: abs(acc) + abs(new), coefficients[:lenCoeff - 1]) / an)

$$\text{Proposición 4.1 } \textit{Sea } p(x):=a_0+...+a_nx^n \textit{ con } a_n > 0 \newline
\textit{y al menos un coeficiente negativo.}\newline
\textit{Sean }\newline
i_0 := \text{máx}\{i,a_i<0\}\newline
M := \text{máx}\{\lvert a_i \rvert, a_i < 0\}\newline
\textit{Si r es un cero positivo de } p(x) \textit{, entonces } r\leq 1 + \sqrt[n-i_0]{\frac{M}{a_n}}

In [117]:
def calculateI_0(coefficients):
    """
    Calcula el valor de i_0 para una lista de coeficientes de un polinomio.

    Args:
        coefficients (list): Lista de coeficientes del polinomio.

    Returns:
        int: Valor de i_0, que representa el máximo índice i tal que a_i < 0.

    Examples:
        >>> coefficients = [2, -1, 3, -4, 0]
        >>> calculateI_0(coefficients)
        3
    """
    positionOfCoeff = 1
    positionOfIndex = 0
    coeffiAndIndex = list(filter(lambda x: x[positionOfCoeff] < 0, zip(range(len(coefficients)), coefficients)))
    i_0 = coeffiAndIndex[-1][positionOfIndex]
    return i_0

def calculateM(coefficients):
    """
    Calcula el valor de M para una lista de coeficientes de un polinomio.

    Args:
        coefficients (list): Lista de coeficientes del polinomio.

    Returns:
        float: Valor de M, que representa el máximo valor absoluto entre los coeficientes negativos.

    Examples:
        >>> coefficients = [2, -1, 3, -4, 0]
        >>> calculateM(coefficients)
        4.0
    """
    return max([abs(x) for x in coefficients if x < 0])


def checkPreconditions(poly: Poly):
    """
    Verifica las precondiciones necesarias para aplicar la fórmula en un polinomio.

    Args:
        poly (Poly): Objeto polinomial.

    Returns:
        bool: True si se cumplen las precondiciones, False en caso contrario.

    Examples:
        >>> p = Poly('3*x**2 - 2*x + 1', domain='QQ')
        >>> checkPreconditions(p)
        True
    """
    coefficients = reverse(poly.all_coeffs())
    a_n = coefficients[-1]
    numOfNegativeCoefficients = len(list(filter(lambda a_i: a_i < 0, coefficients)))
    return (a_n > 0) and (numOfNegativeCoefficients > 0)

def upperBoundPositiveRealRoots(poly: Poly):
    """
    Calcula el límite superior de las raíces reales positivas de un polinomio.

    Args:
        poly (Poly): Objeto polinomial.

    Returns:
        float: Límite superior de las raíces reales positivas.

    Raises:
        Exception: Si el polinomio no cumple con los criterios para determinar el límite superior.

    Examples:
        Al graficar en geogebra podras ver que la raiz positiva mas grande es casi 5
        >>> p = Poly('x**2 - 5*x + 1')
        >>> upperBoundPositiveRealRoots(p)
        6
    """
    if not checkPreconditions(poly):
        raise Exception("The polynomial does not meet the criteria to determine the upper bound.")
    coefficients = reverse(poly.all_coeffs())
    n = poly.degree()
    i_0 = calculateI_0(coefficients)
    a_n = coefficients[-1]
    M = calculateM(coefficients)
    return (M / a_n) ** (1 / (n - i_0)) + 1